# imaport modules

In [ ]:
import os
import cv2
import pandas as pd  
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.metrics import Recall,Precision
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, ResNet152
from tensorflow.keras.layers import Input,concatenate
from tensorflow.keras.models import Model 
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


# Read data

In [ ]:
df=pd.read_csv(r"C:\Users\mwael\OneDrive\Desktop\home\course\Eyes\ODIR-5K\full_df.csv")

In [ ]:
df.head()


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

# EDA

In [ ]:
sns.barplot(data=df,x="labels",y="Patient Age",hue="Patient Sex")


In [ ]:
if 'ID' in df.columns:   
    df=df.drop('ID',axis=1)

In [ ]:
df.columns

In [ ]:
df['Patient Sex'].value_counts()

In [ ]:
plt.figure(figsize=(8,8))
sns.histplot(data=df,x="Patient Age")

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(df['labels'].value_counts(),autopct="%0.1F%%",
        labels=['Normal','Diabetes','Glaucoma','Cataract','Age related Macular Degeneration','Hypertension',
                'Pathological Myopia','Other diseases/abnormalities']
                ,shadow=True,explode=[0.2,0.09,0,0,0,0,0,0]
                )

In [ ]:



sex_counts = df['Patient Sex'].value_counts().reset_index()

sex_counts.columns = ['Sex', 'Count']

plt.figure(figsize=(8,8))
sns.barplot(x='Sex', y='Count', data=sex_counts, color="b", width=0.5)
plt.show()


In [ ]:
plt.figure(figsize=(8,8))
plt.pie(df['Patient Sex'].value_counts(),labels=["MALE","FEMALE"],autopct="%0.1f%%",shadow=True,explode=[0.2,0])

In [ ]:
df.head()

In [ ]:
real_values=["['N']", "['D']", "['O']", "['M']", "['H']", "['C']", "['A']","['G']"]
df['labels'].unique()


# preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels'])


In [ ]:
df['labels'] = df['labels'].astype(str)

In [ ]:
left_images=df[['Left-Fundus','labels']]
Right_images=df[['Right-Fundus','labels']]

left_images.columns=['image','target']
Right_images.columns=['image','target']
left_images

In [ ]:
combined_df=pd.concat([left_images,Right_images])
combined_df = combined_df.dropna(subset=['image'])
combined_df

In [ ]:
train_df,test_df=train_test_split(combined_df,test_size=0.2,random_state=42)
train_df.head()

In [ ]:
datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=r'C:\Users\mwael\OneDrive\Desktop\home\course\Eyes\ODIR-5K\Training Images',
    x_col='image',
    y_col='target',
    subset="training",
    class_mode="categorical",
    target_size=(224,224),
    batch_size=16
)

In [ ]:
validation_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='C:\Users\mwael\OneDrive\Desktop\home\course\Eyes\ODIR-5K\Training Images',
    x_col="image",
    y_col="target",
    # subset="validation",
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32
    )



# Build model stracture & Compile

In [ ]:
input_layer = Input(shape=(224, 224, 3))

resnet50 = ResNet50(weights='imagenet', include_top=False)(input_layer)
flatten_50 = Flatten()(resnet50)

resnet152 = ResNet152(weights='imagenet', include_top=False)(input_layer)
flatten_152 = Flatten()(resnet152)

merged = concatenate([flatten_50, flatten_152])

dense1 = Dense(512, activation='relu')(merged)
dense2 = Dense(256, activation='relu')(dense1)
dense3=Dense(128,activation='relu')(dense2)
dense4=Dense(64,activation='relu')(dense3)
output_layer = Dense(8, activation='softmax')(dense4)

model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=[
                  Recall(name="recall"),
                  Precision(name="Precision"),
                  "accuracy"
                    ])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)


# Train model

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)   


# save model

In [ ]:

model.save('ocular_eyes.keras') 


# validatioin 

In [ ]:
test_generator = datagen.flow_from_dataframe(
    
    dataframe=test_df,
    directory='C:\Users\mwael\OneDrive\Desktop\home\course\Eyes\ODIR-5K\Testing Images',
    x_col="image",
    y_col="target", 
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32,
    shuffle=False)

In [ ]:
results = model.evaluate(test_generator)
print(f'Test Loss: {results[0]}, Test Accuracy: {results[1]}')
